In [6]:
import cv2
import numpy as np
import time

In [7]:
def loadVideo():
    # Laden des Videos
    #video_path = r'C:\Users\mtwar\Documents\MAS Data Science\CAS Machine Intelligence\Deep Learning\LNW\5_Video\WhatsApp Video 2025-05-12 at 11.45.15.mp4'
    #cap = cv2.VideoCapture(video_path)

    # Setze die Startposition auf 15 Sekunden (15000 Millisekunden)
    #start_time_ms = 15000
    #cap.set(cv2.CAP_PROP_POS_MSEC, start_time_ms)

    #video_path = r'C:\Users\mtwar\Documents\MAS Data Science\CAS Machine Intelligence\Deep Learning\LNW\5_Video\chairlift_vid1.mp4'
    video_path = r'C:\Users\mtwar\Documents\MAS Data Science\CAS Machine Intelligence\Deep Learning\LNW\5_Video\WhatsApp Video 2025-05-12 at 11.45.35.mp4'
    cap = cv2.VideoCapture(video_path)
    return cap


In [10]:
cap = loadVideo()

# Erstellen des Hintergrundsubtraktors
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=100, detectShadows=False)

# Original-Framerate des Videos
original_fps = cap.get(cv2.CAP_PROP_FPS)
frame_delay = int(1000 / original_fps)  # Verzögerung in Millisekunden

while cap.isOpened():
    start_time = time.time()
    
    ret, frame = cap.read()
    if not ret:
        break

    # Hintergrundsubtraktion anwenden
    fgmask = fgbg.apply(frame)

    # Morphologische Operationen zur Verbesserung des Masks
    kernel = np.ones((5, 5), np.uint8)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

    # Konturen finden
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) < 500:  # Ignorieren kleiner Konturen
            continue

        # Umgebende Box zeichnen
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Anzeigen des Frames
    cv2.imshow('Frame', frame)
    cv2.imshow('FG Mask', fgmask)

    # Berechnung der Zeit, die für die Verarbeitung benötigt wurde
    processing_time = int((time.time() - start_time) * 1000)

    # Warten, um die ursprüngliche Framerate beizubehalten
    if processing_time < frame_delay:
        cv2.waitKey(frame_delay - processing_time)
    else:
        # Wenn die Verarbeitung länger dauert als die Frame-Zeit, einfach zum nächsten Frame gehen
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [8]:
cap = loadVideo()

# Überprüfen, ob das Video erfolgreich geöffnet wurde
if not cap.isOpened():
    print("Fehler beim Öffnen der Videodatei")
    exit()

# Erstellen des Hintergrundsubtraktors
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=100, detectShadows=False)

# Original-Framerate des Videos
original_fps = cap.get(cv2.CAP_PROP_FPS)
frame_delay = int(1000 / original_fps)  # Verzögerung in Millisekunden

while cap.isOpened():
    start_time = time.time()

    ret, frame = cap.read()
    if not ret:
        break

    # Hintergrundsubtraktion anwenden
    fgmask = fgbg.apply(frame)

    # Morphologische Operationen zur Verbesserung der Maske
    kernel = np.ones((5, 5), np.uint8)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

    # Konturen finden
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Zeichnen der Konturen auf dem Original-Frame
    for contour in contours:
        if cv2.contourArea(contour) < 500:  # Ignorieren kleiner Konturen
            continue

        # Umgebende Box zeichnen
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Anzeigen des Frames mit den erkannten Objekten
    cv2.imshow('Frame with Detected Objects', frame)

    # Berechnung der Zeit, die für die Verarbeitung benötigt wurde
    processing_time = int((time.time() - start_time) * 1000)

    # Warten, um die ursprüngliche Framerate beizubehalten
    if processing_time < frame_delay:
        cv2.waitKey(frame_delay - processing_time)
    else:
        # Wenn die Verarbeitung länger dauert als die Frame-Zeit, einfach zum nächsten Frame gehen
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Freigabe der Ressourcen
cap.release()
cv2.destroyAllWindows()
